In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import json
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from tensorflow.keras.preprocessing import text, sequence
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
beijingAirData = pd.read_csv("C:/Users/Studying/Data/PRSA_Data_Aotizhongxin_20130301-20170228.csv")

In [3]:
conditions = [
    (beijingAirData['PM2.5'] <=60) & (beijingAirData['PM10'] <=100) & (beijingAirData['SO2'] <= 80) & (beijingAirData['NO2'] <=80) & (beijingAirData['O3'] <= 100),
    (beijingAirData['PM2.5'] > 61) | (beijingAirData['PM10'] >101) | (beijingAirData['SO2'] > 81) | (beijingAirData['NO2'] >81) | (beijingAirData['O3'] > 101)
]

values = [1, 2]

beijingAirData['airQuality'] = np.select(conditions, values)

In [4]:
index = beijingAirData[ (beijingAirData['airQuality'] == 0)].index
beijingAirData.drop(index , inplace=True)

updated_beijingAirData = beijingAirData.dropna(axis=0)

In [5]:
features = updated_beijingAirData.drop(labels = ['year', 'month', 'day', 'hour', 'CO', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station', 'airQuality'], axis = 1)
label = updated_beijingAirData.drop(labels = ['No', 'year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station'], axis = 1)

In [6]:
updated_beijingAirData_normal = preprocessing.normalize(features)
updated_beijingAirData_normal = pd.DataFrame(updated_beijingAirData_normal, columns = features.columns)
updated_beijingAirData_normal['airQuality'] = label
updated_beijingAirData_normal.head()

,No,PM2.5,PM10,SO2,NO2,O3,airQuality
0,0.012881,0.051524,0.051524,0.051524,0.090167,0.991837,1.0
1,0.025553,0.102212,0.102212,0.051106,0.089435,0.983790,1.0
2,0.040230,0.093869,0.093869,0.067049,0.134098,0.978918,1.0
3,0.053867,0.080801,0.080801,0.148136,0.148136,0.969615,1.0
4,0.067328,0.040397,0.040397,0.161588,0.161588,0.969527,1.0


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(updated_beijingAirData_normal.drop(labels = ["airQuality"],axis = 1), label, test_size = 0.25, random_state = 0)

1. Повнозв'язані нейронні мережі
Вирішіть завдання класифікації даних, з якими ви працювали в лабораторній № 1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняйте з одержаними раніше. 

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 64)                50240     
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train_categorical = utils.to_categorical(Y_train)
Y_test_categorical = utils.to_categorical(Y_test)

print("Training data shape: ", X_train.shape)
print("Training labels shape: ", Y_train.shape)
print("Test data shape: ", X_test.shape)
print("Test labels shape: ", Y_test.shape)

Training data shape:  (60000, 28, 28)
Training labels shape:  (60000,)
Test data shape:  (10000, 28, 28)
Test labels shape:  (10000,)


In [13]:
history = model.fit(X_train, Y_train_categorical, epochs=10, validation_split=0.33)

Epoch 1/10
1257/1257 [==============================] - 12s 9ms/step - loss: 2.6597 - accuracy: 0.7545 - val_loss: 0.7461 - val_accuracy: 0.8202
Epoch 2/10
1257/1257 [==============================] - 10s 8ms/step - loss: 0.6121 - accuracy: 0.8463 - val_loss: 0.5289 - val_accuracy: 0.8778
Epoch 3/10
1257/1257 [==============================] - 8s 6ms/step - loss: 0.4344 - accuracy: 0.8844 - val_loss: 0.4085 - val_accuracy: 0.8899
Epoch 4/10
1257/1257 [==============================] - 8s 7ms/step - loss: 0.3499 - accuracy: 0.9079 - val_loss: 0.4049 - val_accuracy: 0.8993
Epoch 5/10
1257/1257 [==============================] - 9s 7ms/step - loss: 0.3014 - accuracy: 0.9192 - val_loss: 0.3183 - val_accuracy: 0.9226
Epoch 6/10
1257/1257 [==============================] - 9s 7ms/step - loss: 0.2665 - accuracy: 0.9281 - val_loss: 0.3250 - val_accuracy: 0.9198
Epoch 7/10
1257/1257 [==============================] - 10s 8ms/step - loss: 0.2480 - accuracy: 0.9318 - val_loss: 0.3140 - val_accura

In [14]:
print(history.history.keys())


dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [16]:
train_loss, train_acc = model.evaluate(X_train,  Y_train_categorical)
test_loss, test_acc = model.evaluate(X_test,  Y_test_categorical)
print('Classification accuracy on training set: ', train_acc)
print('Classification accuracy on test set: ', test_acc)

313/313 [==============================] - 1s 5ms/step - loss: 0.2856 - accuracy: 0.9362
Classification accuracy on training set:  0.9445333480834961
Classification accuracy on test set:  0.9362000226974487


In [17]:
test_predict = model.predict(X_test)
test_predict_labels = np.argmax(test_predict, axis=1)
confusion_matrix = tf.math.confusion_matrix(labels=Y_test, predictions=test_predict_labels)
print('Confusion matrix of the test set:\n', confusion_matrix)

313/313 [==============================] - 1s 4ms/step
Confusion matrix of the test set:
 tf.Tensor(
[[ 946    0    0    1    2    0   15    0   13    3]
 [   0 1105    6    2    0    1    6    0   15    0]
 [   3    0  970    3    7    2    8    9   28    2]
 [   5    1   24  870    2   43    1   10   45    9]
 [   2    1    2    0  933    0   14    1    3   26]
 [   2    0    1   17    1  827   13    1   27    3]
 [   2    3    1    0    6   10  926    0   10    0]
 [   3    3   28   10    7    1    0  949    5   22]
 [   2    0    6   12    9   11   16    5  898   15]
 [   7    3    0   11   25    7    0    7   11  938]], shape=(10, 10), dtype=int32)


2. Згорткові нейронні мережі
Вирішіть завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі (якщо в обраному датасеті класів забагато, достатньо залишити 3-5).

In [110]:
dataset_image_gen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

train_data = dataset_image_gen.flow_from_directory("C:/Users/Studying/Data/Dataset_BUSI_with_GT", color_mode="grayscale", batch_size=32, shuffle=True, class_mode='categorical', seed=42,
                                                          subset='training')

valid_data = dataset_image_gen.flow_from_directory("C:/Users/Studying/Data/Dataset_BUSI_with_GT", color_mode="grayscale", batch_size=32, shuffle=True, class_mode='categorical', seed=42,
                                                          subset='validation')

Found 586 images belonging to 3 classes.
Found 194 images belonging to 3 classes.


In [111]:
from tensorflow.keras.utils import image_dataset_from_directory

train_data_gen = image_dataset_from_directory(
                  "C:/Users/Studying/Data/Dataset_BUSI_with_GT",
                  validation_split=0.25,
                  subset="training",
                  seed=42,
                  image_size=(240, 240),
                  batch_size=32)


valid_data_gen = image_dataset_from_directory(
                                       "C:/Users/Studying/Data/Dataset_BUSI_with_GT",
                                        validation_split=0.25,
                                        subset="validation",
                                        seed=42,
                                        image_size=(240,240),
                                        batch_size=32)

Found 780 files belonging to 3 classes.
Using 585 files for training.
Found 780 files belonging to 3 classes.
Using 195 files for validation.


In [112]:
from PIL import Image
import cv2

width, height = Image.open("C:/Users/Studying/Data/Dataset_BUSI_with_GT/normal/normal (130).png").size
print(width*height)
print(width)
print(height)

im = cv2.imread("C:/Users/Studying/Data/Dataset_BUSI_with_GT/normal/normal (130).png")
print(type(im.shape))
print(im.shape)


352503
739
477
<class 'tuple'>
(477, 739, 3)


In [113]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=(240, 240, 3)))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(32, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.4))

model.add(layers.Dense(3, activation='softmax'))

In [114]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_75 (Conv2D)          (None, 238, 238, 16)      448       
                                                                 
 max_pooling2d_73 (MaxPoolin  (None, 119, 119, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_76 (Conv2D)          (None, 117, 117, 32)      4640      
                                                                 
 max_pooling2d_74 (MaxPoolin  (None, 58, 58, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_77 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_75 (MaxPoolin  (None, 28, 28, 64)     

In [115]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=(['accuracy'])
             )

In [116]:
model.compile(optimizer="Adam",
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"])

In [117]:
history = model.fit(train_data_gen,
          batch_size=32,
          epochs=10,
          validation_data=valid_data_gen)



Epoch 1/10
19/19 [==============================] - 17s 802ms/step - loss: 10.7520 - accuracy: 0.4667 - val_loss: 0.9309 - val_accuracy: 0.5333
Epoch 2/10
19/19 [==============================] - 16s 809ms/step - loss: 0.9285 - accuracy: 0.5966 - val_loss: 0.8672 - val_accuracy: 0.5538
Epoch 3/10
19/19 [==============================] - 15s 788ms/step - loss: 0.8108 - accuracy: 0.6410 - val_loss: 0.8678 - val_accuracy: 0.6051
Epoch 4/10
19/19 [==============================] - 15s 751ms/step - loss: 0.6813 - accuracy: 0.7111 - val_loss: 0.8431 - val_accuracy: 0.6564
Epoch 5/10
19/19 [==============================] - 15s 751ms/step - loss: 0.6097 - accuracy: 0.7453 - val_loss: 0.9100 - val_accuracy: 0.6769
Epoch 6/10
19/19 [==============================] - 15s 755ms/step - loss: 0.5278 - accuracy: 0.7932 - val_loss: 0.8944 - val_accuracy: 0.6872
Epoch 7/10
19/19 [==============================] - 16s 851ms/step - loss: 0.4017 - accuracy: 0.8427 - val_loss: 0.8096 - val_accuracy: 0.697

3. Рекурентні нейронні мережі
Вирішіть задачу класифікації текстів (з якими ви працювали в лабораторній № 2) за допомогою рекурентної нейромережі. Результати порівняйте з одержаними раніш. 

In [342]:
data = [json.loads(line) for line in open('C:\\Users\\Studying\\Data2\\reviews_Amazon_Instant_Video.json', 'r')]

In [343]:
reviews = pd.DataFrame(columns = ["reviewText", "overall"])

In [344]:
for i in range(0, 100_000):
    currentItem = data[i]
    reviews.loc[i] = [data[i]["reviewText"], data[i]["overall"]]

In [129]:
counter=0
for item in data:
    counter=counter+1
    
print(counter)

583933


In [345]:
conditions = [
    (reviews['overall'] == 5.0),
    (reviews['overall'] == 1.0) | (reviews['overall'] == 2.0) | (reviews['overall'] == 3.0)]

values = [5, 1]

reviews['overall'] = np.select(conditions, values)

index = reviews[ (reviews['overall'] == 0)].index
reviews.drop(index , inplace=True)

In [346]:
print(reviews.isnull().sum())

reviewText    0
overall       0
dtype: int64


In [349]:
reviews['reviewText'] = reviews['reviewText'].str.lower()
reviews['reviewText'] = reviews['reviewText'].str.replace('[^\w\s]',' ')
reviews['reviewText'] = reviews['reviewText'].str.replace(r'[0-9]', ' ')
reviews['reviewText'] = reviews['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopWords)]))
reviews['reviewText'] = reviews['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word in (words)]))
reviews['reviewText'] = reviews['reviewText'].str.replace('[^\w\s]',' ')
reviews['reviewText'] = reviews['reviewText'].str.replace(' +', ' ')

C:\Users\Studying\AppData\Local\Temp\ipykernel_964\2147008149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['reviewText'] = reviews['reviewText'].str.replace('[^\w\s]',' ')
C:\Users\Studying\AppData\Local\Temp\ipykernel_964\2147008149.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['reviewText'] = reviews['reviewText'].str.replace(r'[0-9]', ' ')
C:\Users\Studying\AppData\Local\Temp\ipykernel_964\2147008149.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['reviewText'] = reviews['reviewText'].str.replace('[^\w\s]',' ')
C:\Users\Studying\AppData\Local\Temp\ipykernel_964\2147008149.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['reviewText'] = reviews['reviewText'].str.replace(' +', ' ')


In [350]:
reviews['reviewText']

0        ca lewsi review removed reviewing episode seas...
1        truly love humor south park social political c...
4        seems like today generation getting revenge ce...
5        dont agree reviewer says show wears thin awhil...
6        apparently one really thinks drawn together fa...
                               ...                        
99994    show capture month old son attention let kid w...
99995    love ygg son loves program creative great mess...
99996    great show fun kids adults refreshing change b...
99997    quite different rest two year old loves laughs...
99999    grandchildren love yo gabba gabba yet single n...
Name: reviewText, Length: 83535, dtype: object

In [425]:
reviews['reviewText'][0]

'ca lewsi review removed reviewing episode season gonna write review whole season episode thats even season reviewing'

In [426]:
reviews['reviewText'][13]

'ok watched least percent drawn together episodes yet ever recall laughing past grin characters boxed stereotypes leaving absolutely nothing funny obvious predictable jokes never displayed wit adult themed animated shows cant see show lasting many seasons slight premise show tired stuff dawg'

In [427]:
reviews['reviewText'][55]

'great purchase thinking buying sets give gifts package arrived time excellent condition still cellophane wrapped series informative enough without feeling like lesson jean simmons voice captivating especially longer us art works unbelievably beautiful fit narrative perfectly bible scholar cannot speak veracity content say enjoyed set even first saw television'

In [409]:
tokenizer = text.Tokenizer(5000)
tokenizer.fit_on_texts(reviews['reviewText'])
reviews_tokenized = tokenizer.texts_to_sequences(reviews['reviewText'])

In [410]:
X_train, X_test, Y_train, Y_test = train_test_split(reviews_tokenized, reviews['overall'], test_size = 0.25, random_state = 0)

In [428]:
max_seq_len = 15

In [429]:
X_train_pad = sequence.pad_sequences(X_train, maxlen=max_seq_len)
X_test_pad = sequence.pad_sequences(X_test, maxlen=max_seq_len)

In [430]:
X_train_pad[67]

array([  91, 1775,  141,    6,  142, 3530,  848,  630,  109,   13, 1775,
        813,  101,   79, 1831])

In [431]:
X_train_pad[55]

array([ 973,    7,  658, 1783,   21,   89,  506,   11,  112,  204,   92,
         37,   67,  696,   20])

In [432]:
print(Y_train)

80980    5
16535    5
8104     5
61228    5
48365    1
        ..
25345    5
54514    5
50592    5
51684    1
81949    1
Name: overall, Length: 62651, dtype: int32


In [433]:
model = Sequential()

model.add(Embedding(5000, 128))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [434]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=(['accuracy']))


In [435]:
model.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_39 (Embedding)    (None, None, 128)         640000    
                                                                 
 lstm_36 (LSTM)              (None, 128)               131584    
                                                                 
 dense_36 (Dense)            (None, 1)                 129       
                                                                 
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [436]:
history = model.fit(X_train_pad,
          Y_train,   
          batch_size=512,
          epochs=10)

Epoch 1/10
123/123 [==============================] - 21s 139ms/step - loss: -59.8548 - accuracy: 0.2158
Epoch 2/10
123/123 [==============================] - 18s 146ms/step - loss: -119.8055 - accuracy: 0.2167
Epoch 3/10
123/123 [==============================] - 19s 152ms/step - loss: -170.5038 - accuracy: 0.2167
Epoch 4/10
123/123 [==============================] - 19s 152ms/step - loss: -220.3515 - accuracy: 0.2167
Epoch 5/10
123/123 [==============================] - 19s 152ms/step - loss: -269.7969 - accuracy: 0.2167
Epoch 6/10
123/123 [==============================] - 19s 152ms/step - loss: -319.0171 - accuracy: 0.2167
Epoch 7/10
123/123 [==============================] - 19s 152ms/step - loss: -368.0743 - accuracy: 0.2167
Epoch 8/10
123/123 [==============================] - 19s 152ms/step - loss: -417.0261 - accuracy: 0.2167
Epoch 9/10
123/123 [==============================] - 19s 151ms/step - loss: -465.9201 - accuracy: 0.2167
Epoch 10/10
123/123 [==========================

Щось тут не так зробив, раніше точніть набагато меншою була, вдалось підвищити до ось цього, але градієнт кудись геть не туди йде(якщо я все правильно зрозумів)